# Proyecto Individual 1 
## *Machine Learning Operations (MLOps)*

### 1. Ingesta de datos

**1.1 Importamos librerías**

In [1]:
#Importamos las librerias que vamos a usar durante el desarrollo del proyecto.

import pandas as pd 
import numpy as np
import json
import gzip
import ast

**1.2 Leemos los datos**

Utilizaremos 3 archivos: steam_games, user_reviews y users_items. 

Mediante el uso de Pandas, leemos el archivo 'steam_games' y lo asignamos a un nombre identificatorio.

In [11]:
#@hidden_cell
ruta1 = '/Users/jpbertone/Documents/Henry Course/Labs/dataPI_MLOps/steam_games.json'
ruta2 = '/Users/jpbertone/Documents/Henry Course/Labs/dataPI_MLOps/user_reviews.json.gz'
ruta3 = '/Users/jpbertone/Documents/Henry Course/Labs/dataPI_MLOps/users_items.json.gz'

1. **steam_games**

In [10]:
df_games = pd.read_json(ruta1, lines=True)

In [12]:
df_games.tail(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,None,Laush Dmitriy Sergeevich,None,NaN,None,NaN
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,None,"xropi,stev3ns",None,NaN,None,NaN
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,NaN,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None,None,None,NaN,None,NaN


Para leer los archivos user_reviews y users_items, utilizamos otro método.

2. **user_reviews**

In [14]:
info = [] # Creamos una lista vacia donde guardaremos los registros del archivo .json

for i in gzip.open(ruta2):
     info.append(ast.literal_eval(i.decode('utf-8')))
     
df_reviews = pd.DataFrame(info) 
  


In [15]:
df_reviews.head(5)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


3. **users_items**

In [16]:
info = [] # Creamos una lista vacia donde guardaremos los registros del archivo .json

for i in gzip.open(ruta3):
     info.append(ast.literal_eval(i.decode('utf-8')))
     
df_items = pd.DataFrame(info) 

In [17]:
df_items.head(5)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


Luego de leer los datos, se observa que los tres dataframes presentan columnas con datos anidados.
 
Para ello, vamos a utilizar las funciones .explode y .json_normalize, con el objetivo de desanidar la informacion y convertir cada clave a columna. 

Del dataframe **df_games** desanidamos la informacion que se encuentra en la columna 'items'.

In [18]:
df_explode = df_games.explode('items')

games_items = pd.json_normalize(df_explode['items'])

In [19]:
games_items.head(5)

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6.0,0.0
1,20,Team Fortress Classic,0.0,0.0
2,30,Day of Defeat,7.0,0.0
3,40,Deathmatch Classic,0.0,0.0
4,50,Half-Life: Opposing Force,0.0,0.0


Del dataframe **df_reviews** desanidamos la informacion que se encuentra en la columna 'reviews'.

In [20]:
df_explode = df_reviews.explode('reviews')

reviews = pd.json_normalize(df_explode['reviews'])

In [21]:
reviews.head(5)

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


Del dataframe **df_items** desanidamos la informacion que se encuentra en la columna 'items'.

In [22]:
df_explode = df_items.explode('items')
user_items = pd.json_normalize(df_explode['items'])

In [24]:
user_items.head(5)

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6.0,0.0
1,20,Team Fortress Classic,0.0,0.0
2,30,Day of Defeat,7.0,0.0
3,40,Deathmatch Classic,0.0,0.0
4,50,Half-Life: Opposing Force,0.0,0.0


In [68]:
df_g = pd.concat([df_games.drop('items', axis=1), games_items], axis=1)

df_g

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,...,id,metascore,developer,user_id,steam_id,items_count,item_id,item_name,playtime_forever,playtime_2weeks
0,None,None,None,None,None,None,None,None,NaN,None,...,NaN,None,None,76561197970982479,7.656120e+16,277.0,10,Counter-Strike,6.0,0.0
1,None,None,None,None,None,None,None,None,NaN,None,...,NaN,None,None,js41637,7.656120e+16,888.0,20,Team Fortress Classic,0.0,0.0
2,None,None,None,None,None,None,None,None,NaN,None,...,NaN,None,None,evcentric,7.656120e+16,137.0,30,Day of Defeat,7.0,0.0
3,None,None,None,None,None,None,None,None,NaN,None,...,NaN,None,None,Riot-Punch,7.656120e+16,328.0,40,Deathmatch Classic,0.0,0.0
4,None,None,None,None,None,None,None,None,NaN,None,...,NaN,None,None,doctr,7.656120e+16,541.0,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5202145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5202146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5202147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5202148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2. Tratamiento de datos

### 3. Feature Engineering

### 4. Disponibilización de datos (API)

### 5. Deployment

### 6. Analisis exploratorio

### 7. Modelamiento